In [2]:
import torch
import finnhub
import json
from datetime import datetime
import os
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
from transformers import pipeline
import yfinance as yf
import pandas as pd

c:\Users\junyanglu\miniconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'yfinance'

In [9]:
generator = pipeline('text-generation', model="facebook/opt-2.7b")
generator("What is Google?")

Device set to use mps:0


[{'generated_text': 'What is Google?\n\nGoogle is a search engine that allows you to search the web.'}]

In [5]:
# load env variables
load_dotenv()

finnhub_api = os.getenv("FINNHUB_API")
finnhub_client = finnhub.Client(api_key=finnhub_api)

Python-dotenv could not parse statement starting at line 2


In [ ]:
if os.path.exists("google_news.json"):
    os.remove("google_news.json")

In [6]:
def preprocess_corpus(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def filter_headlines(news_items, keyword="google"):
    return [item for item in news_items if keyword in item['headline']]

def cosine_similarity_filteration(news_items, threshold=0.8):
    headlines = [item['headline'] for item in news_items]
    vectorizer = TfidfVectorizer().fit_transform(headlines)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    unique_news_items = []
    for i in range(len(news_items)):
        is_unique = True
        for j in range(i):
            if cosine_matrix[i][j] > threshold:
                is_unique = False
                break
        if is_unique:
            unique_news_items.append(news_items[i])
    
    return unique_news_items

In [7]:
# read desired json file
if os.path.exists("test_news.json"):
    with open("test_news.json", "r") as json_file:
        all_news = json.load(json_file)
else:
    all_news = []

date_dict = {
    # "2024-01-01": "2024-01-31",
    # "2024-02-01": "2024-02-28",
    "2024-03-01": "2024-03-31",
    "2024-04-01": "2024-04-30",
    "2024-05-01": "2024-05-31",
    "2024-06-01": "2024-06-30",
    "2024-07-01": "2024-07-31",
    "2024-08-01": "2024-08-31",
}

test_date_dict = {
    "2024-09-01": "2024-09-30",
    "2024-10-01": "2024-10-31",
    "2024-11-01": "2024-11-30",
    "2024-12-01": "2024-12-31"
}

for sd, ed in test_date_dict.items():
    google_news = finnhub_client.company_news('GOOG', _from=sd, to=ed)

    for news_item in google_news:
        # convert unix time to Year-month-day
        news_item['datetime'] = datetime.utcfromtimestamp(news_item['datetime']).strftime('%Y-%m-%d')
        # text preprocessing the headline
        news_item['headline'] = preprocess_corpus(news_item['headline'])

    # keep only the headlines containing the keyword "google"
    google_news = filter_headlines(google_news)

    # Apply cosine similarity filtering
    google_news = cosine_similarity_filteration(google_news)

    all_news.extend(google_news)

# dump all news items to the JSON file
with open("test_news.json", "w") as json_file:
    json.dump(all_news, json_file, indent=4)

print(f"# of news headlines: {len(all_news)}")

# of news headlines: 263


In [10]:
# Fetch stock data using yfinance
symbol = 'GOOG'
start_date = '2024-03-01'
end_date = '2024-08-31'
stock_data = yf.download(symbol, start=start_date, end=end_date)

stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()

stock_data.to_csv('stock_returns.csv')

print(stock_data.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


KeyError: 'Adj Close'